In [1]:
import numpy as np
import pandas as pd
import glob
import os
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

# Benchmarks I Want to Define
- KNN Rent
- KNN Rent by Beds
- KNN Rent by Baths
- KNN Rent by Bed x Bath
- CBSA Average Rent
- CBSA Average Rent by Beds
- CBSA Average Rent by Baths
- CBSA Average Rent by Bed x Bath
- State Average Rent
- State Average Rent by Beds
- State Average Rent by Baths
- State Average Rent by Bed x Bath